# Import Librarys

In [1]:
import project
import pandas                   as pd
import numpy                    as np
import scikitplot               as skplt
from sklearn.model_selection    import StratifiedKFold
from sklearn.ensemble           import GradientBoostingClassifier

# Load the data

In [2]:
cols_selected = ['annual_premium', 'vintage', 'age', 'region_code', 'vehicle_damage', 'previously_insured', 'policy_sales_channel']

df5 = pd.read_csv('../data/dataframe_steps/df5.csv')
x_val = pd.read_csv('../data/dataframe_steps/x_val_pp.csv')

x_train = df5[ cols_selected ]
y_train = df5['response']

x_val1 = x_val[ cols_selected ]
y_val = pd.read_csv('../data/dataframe_steps/y_val.csv')

In [3]:
# append dataframes for cross-validation
cv_x = pd.concat([x_train, x_val1])
cv_y = pd.concat([pd.DataFrame(y_train), y_val])

# dataset for cross validation
dataset = cv_x
dataset['response'] = cv_y

k = int(x_val.shape[0]*0.4)

# GradientBoostingClassifier Model

In [4]:
# model definition
gb_model = GradientBoostingClassifier(random_state=42)
# model training
gb_model.fit( x_train, y_train )

# model prediction
yhat_gb = gb_model.predict_proba( x_val1 )

# Accumulative Gain
skplt.metrics.plot_cumulative_gain( y_val, yhat_gb );

# Lift Curve
skplt.metrics.plot_lift_curve(y_val, yhat_gb);

In [5]:
insurance = project.ProjectFunctions()
gb_score = insurance.make_score(x_val, y_val, yhat_gb)

# compute precision at K
precision_at_n_k = insurance.precision_at_k( gb_score, k=k )
print( f'Precision at K: {precision_at_n_k}' )

# compute recall at K
recall_at_n_k = insurance.recall_at_k( gb_score, k=k )
print( f'Recall    at K: {recall_at_n_k}')

Precision at K: 0.2811505788973072
Recall    at K: 0.9175765360736459


In [6]:
skf = StratifiedKFold(n_splits=10)

model = gb_model

knn_cv = insurance.stratifield_cross_validation( x_data=cv_x, y_data=cv_y,dataset=dataset, skf=skf, model=model, sample_size=0.4)

fold :1 | precision 0.28153 | recall 0.91886
fold :2 | precision 0.28586 | recall 0.93299
fold :3 | precision 0.28062 | recall 0.91586
fold :4 | precision 0.28442 | recall 0.92828
fold :5 | precision 0.28232 | recall 0.92143
fold :6 | precision 0.28055 | recall 0.91565
fold :7 | precision 0.28252 | recall 0.92207
fold :8 | precision 0.28134 | recall 0.91822
fold :9 | precision 0.28114 | recall 0.91758
fold :10 | precision 0.28108 | recall 0.91736
